In [1]:
# This information helps with debugging and getting support :)
import sys, platform
import pandas as pd
import bifacial_radiance as br
print("Working on a ", platform.system(), platform.release())
print("Python version ", sys.version)
print("Pandas version ", pd.__version__)
print("bifacial_radiance version ", br.__version__)

Working on a  Windows 10
Python version  3.9.13 (main, Aug 25 2022, 23:51:50) [MSC v.1916 64 bit (AMD64)]
Pandas version  1.5.3
bifacial_radiance version  0.4.3.dev341+g93e0ec5.d20240830


# 21 - Weather to Module Performance
## Modeling Performance, an End to End Simulation

This tutorial shows how to use the new function on bifacial_radiance calculatePerformanceModule performance, as well as how to find CEC Module parameters.


In [2]:
import os
from pathlib import Path

testfolder = str(Path().resolve().parent.parent / 'bifacial_radiance' / 'TEMP' / 'Tutorial_21')

if not os.path.exists(testfolder): os.mkdir(testfolder)

# Another option using relative address; for some operative systems you might need '/' instead of '\'
# testfolder = os.path.abspath(r'..\..\bifacial_radiance\TEMP')  

print ("Your simulation will be stored in %s" % testfolder)


Your simulation will be stored in C:\Users\cdeline\Documents\Python Scripts\Bifacial_Radiance\bifacial_radiance\TEMP\Tutorial_21


In [3]:
import bifacial_radiance
import numpy as np
import pandas as pd
import pvlib 

bifacial_radiance.__version__

'0.4.3.dev341+g93e0ec5.d20240830'

## Geting a CEC Module to pass into demo.makeModule

In [4]:
url = 'https://raw.githubusercontent.com/NREL/SAM/patch/deploy/libraries/CEC%20Modules.csv'
db = pd.read_csv(url, index_col=0) # Reading this might take 1 min or so, the database is big.

Find the module that you want. In this case we know it's a SunPower of model SPR-E19-310-COM. 

Make sure you select only 1 module from the database -- sometimes there are similar names.

In [5]:
modfilter2 = db.index.str.startswith('SunPower') & db.index.str.endswith('SPR-E19-310-COM')
print(modfilter2)
CECMod = db[modfilter2]
print(len(CECMod), " modules selected. Name of 1st entry: ", CECMod.index[0])

[False False False ... False False False]
1  modules selected. Name of 1st entry:  SunPower SPR-E19-310-COM


In [6]:
# Selecting only two times as examples
starttime = '01_13_11';  endtime = '01_13_12'
demo = bifacial_radiance.RadianceObj('tutorial_21', path = testfolder) # Create a RadianceObj 'object'
weatherfile = demo.getEPW(lat = 37.5, lon = -77.6)  # This location corresponds to Richmond, VA.
metdata = demo.readWeatherFile(weatherFile=weatherfile, starttime=starttime, endtime=endtime)
demo.setGround(0.2)

path = C:\Users\cdeline\Documents\Python Scripts\Bifacial_Radiance\bifacial_radiance\TEMP\Tutorial_21
Getting weather file: USA_VA_Richmond.724010_TMY2.epw
 ... OK!
8760 line in WeatherFile. Assuming this is a standard hourly WeatherFile for the year for purposes of saving Gencumulativesky temporary weather files in EPW folder.
Coercing year to 2021
Filtering dates
Saving file EPWs\metdata_temp.csv, # points: 8760
Calculating Sun position for Metdata that is right-labeled  with a delta of -30 mins. i.e. 12 is 11:30 sunpos
Loading albedo, 1 value(s), 0.200 avg
1 nonzero albedo values.


The CEC data should be passed into the ModuleObj, either at time of creation, or sometime before it is passed into makeScene.

In [7]:
mymodule = demo.makeModule(name='test-module', x=1, y=2, bifi=0.9, CECMod=CECMod) 


Module Name: test-module
Module test-module updated in module.json
Pre-existing .rad file objects\test-module.rad will be overwritten



The same data can be passed after the ModuleObj's definition:

In [8]:
mymodule.addCEC(CECMod)

In [9]:
sceneDict = {'tilt': 10, 'azimuth': 180, 'pitch': 5,'hub_height':1.5, 'nMods':3, 'nRows': 2}
trackerdict = demo.set1axis(metdata = metdata, cumulativesky = False)
trackerdict = demo.gendaylit1axis()
trackerdict = demo.makeScene1axis(trackerdict, module = mymodule, sceneDict = sceneDict)
trackerdict = demo.makeOct1axis()
trackerdict = demo.analysis1axis(sensorsy=3)

Creating ~2 skyfiles. 
Created 2 skyfiles in /skies/

Making ~2 .rad files for gendaylit 1-axis workflow (this takes a minute..)
2 Radfiles created in /objects/

Making 2 octfiles in root directory.
Created 1axis_2021-01-13_1100.oct
Created 1axis_2021-01-13_1200.oct
Linescan in process: 1axis_2021-01-13_1100_Scene0_Row1_Module2_Front
Linescan in process: 1axis_2021-01-13_1100_Scene0_Row1_Module2_Back
Saved: results\irr_1axis_2021-01-13_1100_Scene0_Row1_Module2.csv
Index: 2021-01-13_1100. Wm2Front: 254.17376666666667. Wm2Back: 45.14680666666666
Linescan in process: 1axis_2021-01-13_1200_Scene0_Row1_Module2_Front
Linescan in process: 1axis_2021-01-13_1200_Scene0_Row1_Module2_Back
Saved: results\irr_1axis_2021-01-13_1200_Scene0_Row1_Module2.csv
Index: 2021-01-13_1200. Wm2Front: 253.0892. Wm2Back: 43.43526333333333


## Calculating the Performance and Exporting the Results to a CSV

In [10]:
print(trackerdict)
#tracker_dict_sample = {'2021-01-13_1100':trackerdict['2021-01-13_1100']}
#eff_irr = tracker_dict_sample['Wm2Front'] + tracker_dict_sample['Wm2Back']
Compiled_Results = demo.calculateResults1axis()
#calculatePerformanceModule -> calculcateResults()

{'2021-01-13_1100': {'surf_azm': 90.0, 'surf_tilt': 44.14, 'theta': -44.14, 'dni': 144, 'ghi': 211, 'dhi': 149, 'temp_air': 4.6, 'wind_speed': 3.8, 'skyfile': 'skies\\sky2_37.5_-77.33_2021-01-13_1100.rad', 'scenes': [{'module': {'x': 1, 'y': 2, 'z': 0.02, 'modulematerial': 'black', 'scenex': 1.01, 'sceney': 2.0, 'scenez': 0.1, 'numpanels': 1, 'bifi': 0.9, 'text': '! genbox black test-module 1 2 0.02 | xform -t -0.5 -1.0 0 -a 1 -t 0 2.0 0', 'modulefile': 'objects\\test-module.rad', 'glass': False, 'offsetfromaxis': 0, 'xgap': 0.01, 'ygap': 0.0, 'zgap': 0.1}, 'modulefile': 'objects\\test-module.rad', 'hpc': False, 'name': 'Scene0', 'gcr': 0.4, 'text': '!xform -rx 44.14 -t 0 0 1.5 -a 3 -t 1.01 0 0 -a 2 -t 0 5 0 -i 1 -t -1.01 -0.0 0 -rz 90.0 -t 0 0 0 "objects\\test-module.rad"', 'radfiles': 'objects\\1axis2021-01-13_1100__C_0.80_rtr_5.00_tilt_44_3modsx2rows_origin0,0.rad', 'sceneDict': {'tilt': 44.14, 'azimuth': 90.0, 'pitch': 5, 'nMods': 3, 'nRows': 2, 'modulez': 0.02, 'axis_tilt': 0, 'or

In [11]:
demo.exportTrackerDict(savefile=os.path.join('results','Final_Results.csv'),reindex=False)
pd.read_csv(os.path.join('results','Final_Results.csv'))

Exporting TrackerDict
2


,Unnamed: 0,timestamp,rowNum,modNum,sceneNum,name,Wm2Front,Wm2Back,DNI,DHI,...,theta,surf_tilt,surf_azm,POA_eff,Gfront_mean,Grear_mean,Pout_raw,Mismatch,Pout,Pout_Gfront
0,0,2021-01-13_1100,1,2,0,1axis_2021-01-13_1100_Scene0,[253.6018 254.6151 254.3044],[44.61458 44.93219 45.89365],144,149,...,-44.14,44.14,90.0,294.805893,254.173767,45.146807,93.226676,0.011420,93.216030,80.280919
1,1,2021-01-13_1200,1,2,0,1axis_2021-01-13_1200_Scene0,[252.9921 253.0892 253.1863],[43.26411 43.18745 43.85423],97,200,...,-21.20,21.20,90.0,292.180937,253.089200,43.435263,91.714260,0.005496,91.709219,79.342954
